## PyTorch에서 사용법

- 이미지를 표현하는 데이터 텐서와 정답(label) 초기화

In [1]:
import torch, torchvision

model = torchvision.models.resnet18(pretrained = True)
data = torch.rand(1, 3, 64, 64)       # 1개의 이미지 : 채널 3, 높이 64, 넓이 64
labels = torch.rand(1, 1000)          # 정답의 shape : (1, 1000)

- 순전파 단계

In [2]:
prediction = model(data)     # 순전파 단계(forward pass)

- error를 계산하고 역전파 시행 단계

In [3]:
loss = (prediction - labels).sum()
loss.backward()              # 역전파 단계(backward pass)

- Optimizer에 모델의 모든 매개변수 등록

In [4]:
optim = torch.optim.SGD(model.parameters(), lr = 1e-2, momentum = 0.9)

- 경사하강법(gradient descent) 시작 : Optimizer는 gradient에 따라 각 매개변수 조정

In [5]:
optim.step()                 # 경사하강법(gradient descent) 시작

## Autograd에서 미분(differentiation)

- requires_grad = True를 갖는 2개의 텐서 a와 b 생성

In [6]:
import torch

a = torch.tensor([2., 3.], requires_grad = True)
b = torch.tensor([6., 4.], requires_grad = True)

- a와 b는 신경망의 매개변수, Q는 오차라고 가정

In [7]:
Q = 3 * a ** 3 - b ** 2

- 매개변수들에 대한 오차의 변화도를 구하기 위해 .backward() 호출
- 각 텐서의 .grad 속성에 저장됨

In [8]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient = external_grad)

# 수집된 변화도가 올바른지 확인
print(9 * a ** 2 == a.grad)
print((-2) * b == b.grad)

tensor([True, True])
tensor([True, True])


## DAG에서 제외하기

- 입력 텐서 중 단 하나라도 requires_grad = True를 갖는 경우, 연산의 결과 텐서도 변화도를 갖게 됨

In [9]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad = True)

a = x + y
print(f"Does 'a' require gradients? : {a.requires_grad}")
b = x + z
print(f"Does 'b' require gradients? : {b.requires_grad}")

Does 'a' require gradients? : False
Does 'b' require gradients? : True


- 미리 학습된 모델을 미세조정하는 경우, 일반적으로 모델의 모든 매개변수를 고정하고 분류 계층(classifier layer)만 변경

In [10]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained = True)

# 신경망의 모든 매개변수를 고정
for param in model.parameters():
    param.requires_grad = False

In [11]:
# 고정되지 않은 새로운 선형 계층(linear layer)으로 대체
model.fc = nn.Linear(512, 10)

In [12]:
# 분류기만 최적화
optimizer = optim.SGD(model.parameters(), lr = 1e-2, momentum = 0.9)